In [ ]:
# Remove input cells at runtime (nbsphinx)
import IPython.core.display as d
d.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# Instrument Response Functions (IRFs) and sensitivity

**Recommended datasample(s):**
Datasets of fully-analyzed showers used to obtain Instrument Response Functions, which in the default pipeline workflow are called ``gamma3``, ``proton2`` and ``electron``.

**Data level(s):** DL3 (selected event lists of signal-like events and IRFs)

**Description:**

This notebook contains DL3 and benchmarks for the _protopipe_ pipeline. 
Latest performance results cannot be shown on this public documentation and are therefore hosted at [this RedMine page](https://forge.in2p3.fr/projects/benchmarks-reference-analysis/wiki/Protopipe_performance_data).

**Requirements and steps to reproduce:**

- get a DL3 file generated using ``protopipe-DL3-EventDisplay``,

- execute the notebook with ``protopipe-BENCHMARK``,

``protopipe-BENCHMARK launch --config_file configs/benchmarks.yaml -n DL3/benchmarks_DL3_IRFs_and_sensitivity``

To obtain the list of all available parameters add ``--help-notebook``.

**Comparison with other pipelines:**

The MC production to be used and the appropriate set of files to use for this notebook can be found [here](https://forge.in2p3.fr/projects/step-by-step-reference-mars-analysis/wiki#The-MC-sample ).

**Development and testing:**  

As with any other part of _protopipe_ and being part of the official repository, this notebook can be further developed by any interested contributor.   
The execution of this notebook is not currently automatic, it must be done locally by the user _before_ pushing a pull-request.  
Please, strip the output before pushing.

**TODO:**  
- ...

## Table of contents

* [Optimized cuts](#Optimized-cuts)
    - [Direction](#Direction)
    - [Gamma/Hadron separation](#Gamma/Hadron-separation)
* [Differential sensitivity from cuts optimization](#Differential-sensitivity-from-cuts-optimization)
* [Sensitivity against requirements](#Sensitivity-against-requirements)
* [Sensitivity comparison between pipelines](#Sensitivity-comparison-between-pipelines)
* [IRFs](#IRFs)
    - [Effective area](#Effective-area)
    - [Point Spread Function](#Point-Spread-Function)
        + [Angular resolution](#Angular-resolution)
    - [Energy dispersion](#Energy-dispersion)
        + [Energy resolution](#Energy-resolution)
        + [Energy bias](#Energy-bias)
    - [Background rate](#Background-rate)

## Imports
[back to top](#Table-of-contents)

In [ ]:
# From the standard library
import os
from pathlib import Path

# From pyirf
import pyirf
from pyirf.binning import bin_center
from pyirf.utils import cone_solid_angle

# From other 3rd-party libraries
import numpy as np
import astropy.units as u
from astropy.io import fits
from astropy.table import QTable, Table, Column
import uproot

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.pyplot import rc
import matplotlib.style as style
from cycler import cycler
%matplotlib inline

from protopipe.benchmarks.utils import string_to_boolean, get_fig_size

## Input data
[back to top](#Table-of-contents)

In [ ]:
# Parametrized cell

# Protopipe current
analyses_directory = None # path to the 'analyses' folder
output_directory = Path.cwd() # default output directory for plots
analysis_name = None # Name fo the analysis
input_filename = None # Name of the file produced with protopipe-DL3-EventDisplay
label_protopipe_current = None # If None (default) will be set to analysis name
color_protopipe_current = "Blue"
# Protopipe previous (TODO)
load_protopipe_previous = False
analysis_name_2 = None
input_filename_2 = None
label_protopipe_previous = None # If None (default) will be set to analysis name
color_protopipe_previous = "DarkOrange"
# CTAMARS
load_CTAMARS = True # If True, load CTAMARS data
indir_CTAMARS = None # Input directory for CTAMARS data
MARS_label = None # Plot legend label for CTAMARS
color_CTAMARS = "Red"
# EventDisplay
load_EventDisplay = True # If True, load EventDisplay data
indir_EventDisplay = None # Input directory for EventDisplay data
ED_label = None # Plot legend label for EventDisplay
color_EventDisplay = "Green"
# Requirements
site = 'North' # North (La Palma) or South (Paranal) (default: North)
obs_time = '50h' # Observation time (default: 50 h)
color_requirements = "black"
# Other
use_seaborn = False # If True import seaborn and apply global settings from config file
plots_scale = None

In [ ]:
# Handle boolean variables (papermill reads them as strings)
[load_protopipe_previous, load_CTAMARS,
 load_EventDisplay, use_seaborn] = string_to_boolean([load_protopipe_previous, load_CTAMARS,load_EventDisplay, use_seaborn])

In [ ]:
# Plot aesthetics settings

scale = matplotlib_settings["scale"] if plots_scale is None else float(plots_scale)

style.use(matplotlib_settings["style"])
cmap = matplotlib_settings["cmap"]
rc('font', size=matplotlib_settings["rc"]["font_size"])

#if matplotlib_settings["style"] == "seaborn-colorblind":
    
    # Change color order to have first ones more readable
#    colors_order = ['#0072B2', '#D55E00', '#009E73', '#CC79A7', '#56B4E9', '#F0E442']
#    rc('axes', prop_cycle=cycler(color=colors_order))

if use_seaborn:
    import seaborn as sns
    
    sns.set_theme(context=seaborn_settings["theme"]["context"] if "context" in seaborn_settings["theme"] else "talk",
                  style=seaborn_settings["theme"]["style"] if "style" in seaborn_settings["theme"] else "whitegrid",
                  palette=seaborn_settings["theme"]["palette"] if "palette" in seaborn_settings["theme"] else None,
                  font=seaborn_settings["theme"]["font"] if "font" in seaborn_settings["theme"] else "Fira Sans",
                  font_scale=seaborn_settings["theme"]["font_scale"] if "font_scale" in seaborn_settings["theme"] else 1.0,
                  color_codes=seaborn_settings["theme"]["color_codes"] if "color_codes" in seaborn_settings["theme"] else True
                  )
    
    sns.set_style(seaborn_settings["theme"]["style"], rc=seaborn_settings["rc_style"])
    sns.set_context(seaborn_settings["theme"]["context"],
                    font_scale=seaborn_settings["theme"]["font_scale"] if "font_scale" in seaborn_settings["theme"] else 1.0)

if matplotlib_settings["style"] == "seaborn-colorblind":
    
    # Change color order to have first ones more readable
    # here we specify the colors to the data since not all axes have the same data
    
    color_requirements = "black"
    color_protopipe_current = '#0072B2'
    color_EventDisplay = '#D55E00'
    color_CTAMARS = '#009E73'
    color_protopipe_previous = '#CC79A7'

In [ ]:
# First we check if a _plots_ folder exists already.  
# If not, we create it.
plots_folder = Path(output_directory) / "plots"
plots_folder.mkdir(parents=True, exist_ok=True)

### Protopipe
[back to top](#Table-of-contents)

In [ ]:
if input_filename is None:
    try:
        input_filename = input_filenames["DL3"]
    except (NameError, KeyError):
        raise ValueError("The name of the input file is undefined: please use benchmarks.yaml or define it using the CLI.")

production = input_filename.split("protopipe_")[1].split("_Time")[0]
input_directory = Path(analyses_directory) / analysis_name / Path("data/DL3")

protopipe_file = input_directory / input_filename
if not label_protopipe_current:
    label_protopipe_current = f"protopipe {analysis_name}"

In [ ]:
if load_protopipe_previous:

    if input_filename_2 is None:
        raise ValueError("The name of the input file for the second analysis is undefined: please define it using the CLI.")

    production = input_filename_2.split("protopipe_")[1].split("_Time")[0]
    input_directory_2 = Path(analyses_directory) / analysis_name_2 / Path("data/DL3")

    protopipe_file_2 = input_directory_2 / input_filename_2
    if not label_protopipe_previous:
        label_protopipe_previous = f"protopipe {analysis_name_2}"

### ASWG performance
[back to top](#Table-of-contents)

In [ ]:
if load_CTAMARS:
    if indir_CTAMARS is None:
        try:
            indir_CTAMARS = Path(input_data_CTAMARS["parent_directory"]) / Path(input_data_CTAMARS["DL3"]["input_directory"])
            infile_CTAMARS = input_data_CTAMARS["DL3"]["input_file"]
            MARS_performance = uproot.open(Path(indir_CTAMARS) / infile_CTAMARS)
            MARS_label = input_data_CTAMARS["label"]
        except (NameError, KeyError):
            raise ValueError("CTAMARS data is undefined.")

In [ ]:
if load_EventDisplay:
    try:
        indir_ED = Path(input_data_EventDisplay["input_directory"])
        infile_ED = input_data_EventDisplay["input_file"]
        ED_performance = uproot.open(Path(indir_ED) / infile_ED)
        ED_label = input_data_EventDisplay["label"]
    except (NameError, KeyError):
        raise ValueError("EventDisplay data is undefined.")

### Requirements
[back to top](#Table-of-contents)

In [ ]:
if load_requirements:
    try:
        requirements_indir=requirements_input_directory
    except (NameError, KeyError):
        print("WARNING: Requirements data undefined! Please, check the documentation of protopipe-BENCHMARKS.")

infiles = dict(sens=f'/{site}-{obs_time}.dat')

requirements_input_filenames = {"sens" : f'/{site}-{obs_time}.dat',
                                    "AngRes" : f'/{site}-{obs_time}-AngRes.dat',
                                    "ERes" : f'/{site}-{obs_time}-ERes.dat'}
requirements = {}

for key in requirements_input_filenames.keys():
    requirements[key] = Table.read(requirements_input_directory + requirements_input_filenames[key], format='ascii')
requirements['sens'].add_column(Column(data=(10**requirements['sens']['col1']), name='ENERGY'))
requirements['sens'].add_column(Column(data=requirements['sens']['col2'], name='SENSITIVITY'))

## Optimized cuts
[back to top](#Table-of-contents)

### Direction
[back to top](#Table-of-contents)

In [ ]:
plt.figure(figsize=get_fig_size(ratio=4./3, scale=scale))

# protopipe
rad_max = QTable.read(protopipe_file, hdu='RAD_MAX')[0]
plt.errorbar(
    0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI'])[1:-1].to_value(u.TeV),
    rad_max['RAD_MAX'].T[1:-1, 0].to_value(u.deg),
    xerr=0.5 * (rad_max['ENERG_HI'] - rad_max['ENERG_LO'])[1:-1].to_value(u.TeV),
    ls='',
    label=label_protopipe_current,
    color=color_protopipe_current
)

# protopipe previous
if load_protopipe_previous:
    rad_max_2 = QTable.read(protopipe_file_2, hdu='RAD_MAX')[0]
    plt.errorbar(
        0.5 * (rad_max_2['ENERG_LO'] + rad_max_2['ENERG_HI'])[1:-1].to_value(u.TeV),
        rad_max_2['RAD_MAX'].T[1:-1, 0].to_value(u.deg),
        xerr=0.5 * (rad_max_2['ENERG_HI'] - rad_max_2['ENERG_LO'])[1:-1].to_value(u.TeV),
        ls='',
        label=label_protopipe_previous,
        color=color_protopipe_previous
    )

# ED
if load_EventDisplay:
    theta_cut_ed, edges = ED_performance['ThetaCut;1'].to_numpy()
    plt.errorbar(
        bin_center(10**edges),
        theta_cut_ed,
        xerr=np.diff(10**edges),
        ls='',
        label=ED_label,
        color=color_EventDisplay
    )

# MARS
if load_CTAMARS:
    theta_cut_ed = np.sqrt(MARS_performance['Theta2Cut;1'].to_numpy()[0])
    edges = MARS_performance['Theta2Cut;1'].to_numpy()[1]
    plt.errorbar(
        bin_center(10**edges),
        theta_cut_ed,
        xerr=np.diff(10**edges),
        ls='',
        label=MARS_label,
        color=color_CTAMARS
    )

plt.legend()
plt.ylabel('Direction cut [deg]')
plt.xlabel('Reconstructed energy [TeV]')
plt.xscale('log')
plt.grid(visible=True)

None # to remove clutter by mpl objects

### Gamma/Hadron separation
[back to top](#Table-of-contents)

In [ ]:
plt.figure(figsize=get_fig_size(ratio=4./3, scale=scale))

# protopipe
gh_cut = QTable.read(protopipe_file, hdu='GH_CUTS')[1:-1]
plt.errorbar(
    0.5 * (gh_cut['low'] + gh_cut['high']).to_value(u.TeV),
    gh_cut['cut'],
    xerr=0.5 * (gh_cut['high'] - gh_cut['low']).to_value(u.TeV),
    ls='',
    label=label_protopipe_current,
    color=color_protopipe_current
)

if load_protopipe_previous:

    gh_cut_2 = QTable.read(protopipe_file_2, hdu='GH_CUTS')[1:-1]
    plt.errorbar(
        0.5 * (gh_cut_2['low'] + gh_cut_2['high']).to_value(u.TeV),
        gh_cut_2['cut'],
        xerr=0.5 * (gh_cut_2['high'] - gh_cut_2['low']).to_value(u.TeV),
        ls='',
        label=label_protopipe_previous,
        color=color_protopipe_previous
    )
    plt.legend()

plt.ylabel('gamma/hadron cut')
plt.xlabel('Reconstructed energy [TeV]')
plt.xscale('log')
plt.ylim(-0.25,1.25)
plt.grid(visible=True)

None # to remove clutter by mpl objects

## Differential sensitivity from cuts optimization
[back to top](#Table-of-contents)

In [ ]:
# [1:-1] removes under/overflow bins
sensitivity_protopipe = QTable.read(protopipe_file, hdu='SENSITIVITY')[1:-1]

# make it print nice
sensitivity_protopipe['reco_energy_low'].info.format = '.3g'
sensitivity_protopipe['reco_energy_high'].info.format = '.3g'
sensitivity_protopipe['reco_energy_center'].info.format = '.3g'
sensitivity_protopipe['relative_sensitivity'].info.format = '.2g'
sensitivity_protopipe['flux_sensitivity'].info.format = '.3g'

for k in filter(lambda k: k.startswith('n_'), sensitivity_protopipe.colnames):
    sensitivity_protopipe[k].info.format = '.1f'

print(sensitivity_protopipe)

if load_protopipe_previous:
    # [1:-1] removes under/overflow bins
    sensitivity_protopipe_2 = QTable.read(protopipe_file_2, hdu='SENSITIVITY')[1:-1]
    
    # make it print nice
    sensitivity_protopipe_2['reco_energy_low'].info.format = '.3g'
    sensitivity_protopipe_2['reco_energy_high'].info.format = '.3g'
    sensitivity_protopipe_2['reco_energy_center'].info.format = '.3g'
    sensitivity_protopipe_2['relative_sensitivity'].info.format = '.2g'
    sensitivity_protopipe_2['flux_sensitivity'].info.format = '.3g'

    for k in filter(lambda k: k.startswith('n_'), sensitivity_protopipe_2.colnames):
        sensitivity_protopipe_2[k].info.format = '.1f'
        
    print(sensitivity_protopipe_2)

## Sensitivity against requirements
[back to top](#Table-of-contents)

In [ ]:
if load_requirements:
    
    plt.figure(figsize=get_fig_size(ratio=16./9, scale=scale))

    unit = u.Unit('erg cm-2 s-1')

    # protopipe
    e = sensitivity_protopipe['reco_energy_center']
    w = (sensitivity_protopipe['reco_energy_high'] - sensitivity_protopipe['reco_energy_low'])
    s = (e**2 * sensitivity_protopipe['flux_sensitivity'])

    plt.errorbar(
        e.to_value(u.TeV),
        s.to_value(unit),
        xerr=w.to_value(u.TeV) / 2,
        ls='',
        label=label_protopipe_current,
        color=color_protopipe_current
    )
    
    if load_protopipe_previous:
        e_2 = sensitivity_protopipe_2['reco_energy_center']
        w_2 = (sensitivity_protopipe_2['reco_energy_high'] - sensitivity_protopipe_2['reco_energy_low'])
        s_2 = (e_2**2 * sensitivity_protopipe_2['flux_sensitivity'])

        plt.errorbar(
            e_2.to_value(u.TeV),
            s_2.to_value(unit),
            xerr=w_2.to_value(u.TeV) / 2,
            ls='',
            label=label_protopipe_previous,
            color=color_protopipe_previous
        )

    # Add requirements
    plt.plot(requirements['sens']['ENERGY'], 
             requirements['sens']['SENSITIVITY'], 
             color=color_requirements, 
             ls='--', 
             lw=2, 
             label='Requirements'
    )

    # Style settings
    plt.title(f'Minimal Flux Satisfying Requirements for {obs_time} \n {production}')
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(1.e-2, 3.e2)
    plt.ylim(5e-14, 5e-10)
    plt.ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")
    plt.xlabel("Reco Energy [TeV]")

    plt.grid(which="both")
    plt.legend()

else:
    print("Requirement data has not been loaded.")

None # to remove clutter by mpl objects

## Sensitivity comparison between pipelines
[back to top](#Table-of-contents)

In [ ]:
if any([load_protopipe_previous, load_CTAMARS, load_EventDisplay]):
    fig, ax_sens = plt.subplots( figsize=get_fig_size(ratio=16./9, scale=scale) )
    unit = u.Unit('erg cm-2 s-1')

    if load_requirements:
        ax_sens.plot(requirements['sens']['ENERGY'], 
                 requirements['sens']['SENSITIVITY'], 
                 color=color_requirements, 
                 ls='--', 
                 lw=2, 
                 label='Requirements'
        )

    # protopipe
    e = sensitivity_protopipe['reco_energy_center']
    w = (sensitivity_protopipe['reco_energy_high'] - sensitivity_protopipe['reco_energy_low'])
    s_p = (e**2 * sensitivity_protopipe['flux_sensitivity'])
    ax_sens.errorbar(
        e.to_value(u.TeV),
        s_p.to_value(unit),
        xerr=w.to_value(u.TeV) / 2,
        ls='',
        label=label_protopipe_current,
        color=color_protopipe_current
    )
    
    if load_protopipe_previous:
        e_2 = sensitivity_protopipe_2['reco_energy_center']
        w_2 = (sensitivity_protopipe_2['reco_energy_high'] - sensitivity_protopipe_2['reco_energy_low'])
        s_p_2 = (e_2**2 * sensitivity_protopipe_2['flux_sensitivity'])
        ax_sens.errorbar(
            e_2.to_value(u.TeV),
            s_p_2.to_value(unit),
            xerr=w_2.to_value(u.TeV) / 2,
            ls='',
            label=label_protopipe_previous,
            color=color_protopipe_previous
        )

    # ED
    if load_EventDisplay:
        s_ED, edges = ED_performance["DiffSens"].to_numpy()
        yerr = ED_performance["DiffSens"].errors()
        bins = 10**edges
        x = bin_center(bins)
        width = np.diff(bins)
        ax_sens.errorbar(
            x,
            s_ED, 
            xerr=width/2,
            yerr=yerr,
            label=ED_label,
            ls='',
            color=color_EventDisplay
        )

    # MARS
    if load_CTAMARS:
        s_MARS, edges = MARS_performance["DiffSens"].to_numpy()
        yerr = MARS_performance["DiffSens"].errors()
        bins = 10**edges
        x = bin_center(bins)
        width = np.diff(bins)
        ax_sens.errorbar(
            x,
            s_MARS, 
            xerr=width/2,
            yerr=yerr,
            label=MARS_label,
            ls='',
            color=color_CTAMARS
        )

    # Style settings
    ax_sens.set_title(f'Minimal Flux Satisfying Requirements for {obs_time} \n {production}')
    ax_sens.set_xscale("log")
    ax_sens.set_yscale("log")
    ax_sens.set_ylabel(fr"$E^2 \cdot \mathrm{{Flux Sensitivity}} $ [{unit.to_string('latex')}]")
    ax_sens.set_xlabel("Reconstructed energy [TeV]")
    ax_sens.set_xlim(1.e-2, 3.e2)
    ax_sens.set_ylim(5e-14, 5e-10)
    ax_sens.grid(which="both")
    ax_sens.legend()

else:
    print("No reference data from other pipelines was loaded.")

None # to remove clutter by mpl objects

In [ ]:
if any([load_protopipe_previous, load_CTAMARS, load_EventDisplay]):
    fig, (ax_sens, ax_ratio) = plt.subplots(
        2, 1,
        gridspec_kw={'height_ratios': [4, 1]},
        sharex=True,
        figsize=get_fig_size(ratio=4./3, scale=scale)
    )
    unit = u.Unit('erg cm-2 s-1')

    if load_requirements:
        ax_sens.plot(requirements['sens']['ENERGY'], 
                 requirements['sens']['SENSITIVITY'], 
                 color=color_requirements, 
                 ls='--', 
                 lw=2, 
                 label='Requirements'
        )

    # protopipe
    e = sensitivity_protopipe['reco_energy_center']
    w = (sensitivity_protopipe['reco_energy_high'] - sensitivity_protopipe['reco_energy_low'])
    s_p = (e**2 * sensitivity_protopipe['flux_sensitivity'])
    ax_sens.errorbar(
        e.to_value(u.TeV),
        s_p.to_value(unit),
        xerr=w.to_value(u.TeV) / 2,
        ls='',
        label=label_protopipe_current,
        color=color_protopipe_current
    )
    
    if load_protopipe_previous:
        e_2 = sensitivity_protopipe_2['reco_energy_center']
        w_2 = (sensitivity_protopipe_2['reco_energy_high'] - sensitivity_protopipe_2['reco_energy_low'])
        s_p_2 = (e_2**2 * sensitivity_protopipe_2['flux_sensitivity'])
        ax_sens.errorbar(
            e_2.to_value(u.TeV),
            s_p_2.to_value(unit),
            xerr=w_2.to_value(u.TeV) / 2,
            ls='',
            label=label_protopipe_previous,
            color=color_protopipe_previous
        )
        ax_ratio.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_p_2.to_value(unit),
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_protopipe_previous
        )

    # ED
    if load_EventDisplay:
        s_ED, edges = ED_performance["DiffSens"].to_numpy()
        yerr = ED_performance["DiffSens"].errors()
        bins = 10**edges
        x = bin_center(bins)
        width = np.diff(bins)
        ax_sens.errorbar(
            x,
            s_ED, 
            xerr=width/2,
            yerr=yerr,
            label=ED_label,
            ls='',
            color=color_EventDisplay
        )
        ax_ratio.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_ED,
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_EventDisplay
        )

    # MARS
    if load_CTAMARS:
        s_MARS, edges = MARS_performance["DiffSens"].to_numpy()
        yerr = MARS_performance["DiffSens"].errors()
        bins = 10**edges
        x = bin_center(bins)
        width = np.diff(bins)
        ax_sens.errorbar(
            x,
            s_MARS, 
            xerr=width/2,
            yerr=yerr,
            label=MARS_label,
            ls='',
            color=color_CTAMARS
        )

        ax_ratio.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_MARS,
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_CTAMARS
        )

    ax_ratio.axhline(1, color = color_protopipe_current)

    ax_ratio.set_yscale('log')
    ax_ratio.set_xlabel("Reconstructed energy [TeV]")
    ax_ratio.set_ylabel('Ratio')
    ax_ratio.grid()
    ax_ratio.yaxis.set_major_formatter(ScalarFormatter())

    ax_ratio.set_ylim(0.5, 3.0)
    ax_ratio.set_yticks([0.5, 2/3, 1, 3/2, 2])
    ax_ratio.set_yticks([], minor=True)

    # Style settings
    ax_sens.set_title(f'Minimal Flux Satisfying Requirements for {obs_time} \n {production}')
    ax_sens.set_xscale("log")
    ax_sens.set_yscale("log")
    ax_sens.set_ylabel(rf"$E^2 \cdot \mathrm{{Flux Sensitivity}} $ [{unit.to_string('latex')}]")

    ax_sens.grid(which="both")
    ax_sens.legend()
    fig.tight_layout(h_pad=0)

else:
    print("No reference data from other pipelines was loaded.")

None # to remove clutter by mpl objects

## IRFs
[back to top](#Table-of-contents)

### Effective area
[back to top](#Table-of-contents)

In [ ]:
plt.figure(figsize=get_fig_size(ratio=1,scale=scale))

# protopipe
# uncomment the other strings to see effective areas
# for the different cut levels. Left out here for better
# visibility of the final effective areas.
suffix =''
#'_NO_CUTS'
#'_ONLY_GH'
#'_ONLY_THETA'

area = QTable.read(protopipe_file, hdu='EFFECTIVE AREA' + suffix)[0]
plt.errorbar(
    0.5 * (area['ENERG_LO'] + area['ENERG_HI']).to_value(u.TeV)[1:-1],
    area['EFFAREA'].to_value(u.m**2).T[1:-1, 0],
    xerr=0.5 * (area['ENERG_LO'] - area['ENERG_HI']).to_value(u.TeV)[1:-1],
    ls='',
    label=label_protopipe_current + suffix,
    color=color_protopipe_current
)

# protopipe previous
if load_protopipe_previous:
    area_2 = QTable.read(protopipe_file_2, hdu='EFFECTIVE AREA' + suffix)[0]
    plt.errorbar(
        0.5 * (area_2['ENERG_LO'] + area_2['ENERG_HI']).to_value(u.TeV)[1:-1],
        area_2['EFFAREA'].to_value(u.m**2).T[1:-1, 0],
        xerr=0.5 * (area_2['ENERG_LO'] - area_2['ENERG_HI']).to_value(u.TeV)[1:-1],
        ls='',
        label=label_protopipe_previous + suffix,
        color=color_protopipe_previous
    )
    
# ED
if load_EventDisplay:
    y, edges = ED_performance["EffectiveAreaEtrue"].to_numpy()
    yerr = ED_performance["EffectiveAreaEtrue"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=ED_label,
                 color=color_EventDisplay
                )

# MARS
if load_CTAMARS:
    y, edges = MARS_performance["EffectiveAreaEtrue"].to_numpy()
    yerr = MARS_performance["EffectiveAreaEtrue"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=MARS_label,
                 color=color_CTAMARS
                )

# Style settings
plt.xscale("log")
plt.yscale("log")
plt.xlim(1.e-2, 3.e2)
plt.ylim(1.e1, 5.e6)
plt.xlabel("True energy [TeV]")
plt.ylabel("Effective collection area [m²]")
plt.grid(which="both", visible=True)
plt.legend(loc="lower right")

None # to remove clutter by mpl objects

### Point Spread Function
[back to top](#Table-of-contents)

In [ ]:
if load_protopipe_previous:
    psf_table_2 = QTable.read(protopipe_file_2, hdu='PSF')[0]
    # select the only fov offset bin
    psf_2 = psf_table_2['RPSF'].T[:, 0, :].to_value(1 / u.sr)

    offset_bins_2 = np.append(psf_table_2['RAD_LO'], psf_table_2['RAD_HI'][-1])

In [ ]:
psf_table = QTable.read(protopipe_file, hdu='PSF')[0]
# select the only fov offset bin
psf = psf_table['RPSF'].T[:, 0, :].to_value(1 / u.sr)

offset_bins = np.append(psf_table['RAD_LO'], psf_table['RAD_HI'][-1])
phi_bins = np.linspace(0, 2 * np.pi, 100)



# Let's make a nice 2d representation of the radially symmetric PSF
r, phi = np.meshgrid(offset_bins.to_value(u.deg), phi_bins)

# look at a single energy bin
# repeat values for each phi bin
center = 0.5 * (psf_table['ENERG_LO'] + psf_table['ENERG_HI'])


fig = plt.figure(figsize=get_fig_size(ratio=16/9., scale=scale))
plt.suptitle(production)
axs = [fig.add_subplot(1, 3, i, projection='polar') for i in range(1, 4)]


for bin_id, ax in zip([10, 20, 30], axs):
    image = np.tile(psf[bin_id], (len(phi_bins) - 1, 1))
    
    ax.set_title(f'PSF @ {center[bin_id]:.2f}')
    ax.pcolormesh(phi, r, image)
    ax.set_ylim(0, 0.25)
    ax.set_aspect(1)
    
fig.tight_layout()

None # to remove clutter by mpl objects


In [ ]:
fig = plt.figure(figsize=get_fig_size(ratio=4/3.))

bins_to_plot = [10, 20, 30]

# Profile
center = 0.5 * (offset_bins[1:] + offset_bins[:-1])
xerr = 0.5 * (offset_bins[1:] - offset_bins[:-1])

for bin_id in bins_to_plot:
    plt.errorbar(
        center.to_value(u.deg),
        psf[bin_id],
        xerr=xerr.to_value(u.deg),
        ls='',
        label=f'Energy Bin {bin_id} - {analysis_name}'
    )

if load_protopipe_previous:
    # Profile
    center_2 = 0.5 * (offset_bins_2[1:] + offset_bins_2[:-1])
    xerr_2 = 0.5 * (offset_bins_2[1:] - offset_bins_2[:-1])

    for bin_id in bins_to_plot:
        plt.errorbar(
            center_2.to_value(u.deg),
            psf_2[bin_id],
            xerr=xerr_2.to_value(u.deg),
            ls='',
            label=f'Energy Bin {bin_id} - {analysis_name_2}'
        )
    
#plt.yscale('log')
plt.legend()
plt.xlim(0, 0.25)
plt.ylabel('PSF PDF [sr⁻¹]')
plt.xlabel('Distance from True Source [deg]')
plt.title(production)
plt.grid(visible=True)

None # to remove clutter by mpl objects

#### Angular resolution
[back to top](#Table-of-contents)

**NOTE:** for the comparison with CTAMARS and EventDisplay, angular resolutions are plotted as a function of reconstructed energy.

In [ ]:
fig = plt.figure(figsize=get_fig_size(ratio=4/3., scale=scale))

# protopipe
ang_res = QTable.read(protopipe_file, hdu='ANGULAR_RESOLUTION')

plt.errorbar(
    0.5 * (ang_res['reco_energy_low'] + ang_res['reco_energy_high']).to_value(u.TeV),
    ang_res['angular_resolution'].to_value(u.deg),
    xerr=0.5 * (ang_res['reco_energy_high'] - ang_res['reco_energy_low']).to_value(u.TeV),
    ls='',
    label=label_protopipe_current,
    color=color_protopipe_current
)

if load_protopipe_previous:
    ang_res_2 = QTable.read(protopipe_file_2, hdu='ANGULAR_RESOLUTION')

    plt.errorbar(
        0.5 * (ang_res_2['reco_energy_low'] + ang_res_2['reco_energy_high']).to_value(u.TeV),
        ang_res_2['angular_resolution'].to_value(u.deg),
        xerr=0.5 * (ang_res_2['reco_energy_high'] - ang_res_2['reco_energy_low']).to_value(u.TeV),
        ls='',
        label=label_protopipe_previous,
        color=color_protopipe_previous
    )

# ED
if load_EventDisplay:
    y, edges = ED_performance["AngRes"].to_numpy()
    yerr = ED_performance["AngRes"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=ED_label,
                 color=color_EventDisplay)

# MARS
if load_CTAMARS:
    y, edges = MARS_performance["AngRes"].to_numpy()
    yerr = MARS_performance["AngRes"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=MARS_label,
                 color=color_CTAMARS)

# Requirements
if load_requirements:
    plt.plot(10**requirements['AngRes']['col1'], 
             requirements['AngRes']['col2'], 
             color=color_requirements, 
             ls='--', 
             lw=2, 
             label='Requirements'
    )

# Style settings
plt.xscale("log")
plt.yscale("log")
plt.xlim(1.e-2, 3.e2)
plt.ylim(1.e-2, 1.e0)
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Angular Resolution [deg]")
plt.title(production)
plt.grid(which="both")
plt.legend(loc="best")

None # to remove clutter by mpl objects

### Energy dispersion
[back to top](#Table-of-contents)

In [ ]:
from matplotlib.colors import LogNorm

if load_EventDisplay:
    
    fig = plt.figure(figsize=get_fig_size(ratio=16/9., scale=scale))
    plt.subplots_adjust(wspace=0.4)

    plt.subplot(1,2,1)
    plt.title(label_protopipe_current)
else:
    fig = plt.figure(figsize=get_fig_size(ratio=4/3., scale=scale))

edisp = QTable.read(protopipe_file, hdu='ENERGY_DISPERSION')[0]

e_bins = edisp['ENERG_LO'][1:]
migra_bins = edisp['MIGRA_LO'][1:]
    
plt.pcolormesh(e_bins.to_value(u.TeV), 
               migra_bins, 
               edisp['MATRIX'].T[1:-1, 1:-1, 0].T,
               cmap=cmap,
               norm=LogNorm())

plt.xscale('log')
plt.grid(visible=True)
plt.colorbar(label='PDF Value')

plt.xlabel("True energy [TeV]")
plt.ylabel("Reconstructed energy / True energy")

if load_EventDisplay:
    
    plt.subplot(1,2,2)
    plt.title(ED_label)
    ED_migra, X, Y = ED_performance["EestOverEtrue"].to_numpy()
    # make PDF
    ED_migra = ED_migra/np.sum(ED_migra, axis=1)[np.newaxis].T

    plt.pcolormesh(X, 
                   Y, 
                   ED_migra.T,
                   cmap=cmap,
                   norm=LogNorm())
    plt.grid(visible=True)
    plt.colorbar(label='PDF Value')

    plt.xlabel("True energy [TeV]")
    plt.ylabel("Reconstructed energy / True energy")

plt.xlim(-2,2)
plt.ylim(0,4)
    

None # to remove clutter by mpl objects

#### Energy resolution
[back to top](#Table-of-contents)

In [ ]:
fig = plt.figure(figsize=get_fig_size(ratio=4/3., scale=scale))

# protopipe
bias_resolution = QTable.read(protopipe_file, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
plt.errorbar(
    0.5 * (bias_resolution['reco_energy_low'] + bias_resolution['reco_energy_high']).to_value(u.TeV),
    bias_resolution['resolution'],
    xerr=0.5 * (bias_resolution['reco_energy_high'] - bias_resolution['reco_energy_low']).to_value(u.TeV),
    ls='',
    label=label_protopipe_current,
    color=color_protopipe_current
)

if load_protopipe_previous:
    bias_resolution_2 = QTable.read(protopipe_file_2, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
    plt.errorbar(
        0.5 * (bias_resolution_2['reco_energy_low'] + bias_resolution_2['reco_energy_high']).to_value(u.TeV),
        bias_resolution_2['resolution'],
        xerr=0.5 * (bias_resolution_2['reco_energy_high'] - bias_resolution_2['reco_energy_low']).to_value(u.TeV),
        ls='',
        label=label_protopipe_previous,
        color=color_protopipe_previous
    )


# ED
if load_EventDisplay:
    y, edges = ED_performance["ERes"].to_numpy()
    yerr = ED_performance["ERes"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=ED_label,
                 color=color_EventDisplay
                )

# MARS
if load_CTAMARS:
    y, edges = MARS_performance["ERes"].to_numpy()
    yerr = MARS_performance["ERes"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=MARS_label,
                 color=color_CTAMARS
                )

# Requirements
if load_requirements:
    plt.plot(10**requirements['ERes']['col1'], 
             requirements['ERes']['col2'], 
             color=color_requirements, 
             ls='--',
             lw=2, 
             label='Requirements'
    )

# Style settings
plt.xscale('log')
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Energy resolution")
plt.grid(which="both", axis="both", visible=True)
plt.legend(loc="best")
plt.title(production)

None # to remove clutter by mpl objects

#### Energy bias
[back to top](#Table-of-contents)

In [ ]:
fig = plt.figure(figsize=get_fig_size(ratio=4/3., scale=scale))

# protopipe
bias_resolution = QTable.read(protopipe_file, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
plt.errorbar(
    0.5 * (bias_resolution['reco_energy_low'] + bias_resolution['reco_energy_high']).to_value(u.TeV),
    bias_resolution['bias']+1,  # not sure ...
    xerr=0.5 * (bias_resolution['reco_energy_high'] - bias_resolution['reco_energy_low']).to_value(u.TeV),
    ls='',
    label=label_protopipe_current,
    color=color_protopipe_current
)

if load_protopipe_previous:
    bias_resolution_2 = QTable.read(protopipe_file_2, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
    plt.errorbar(
        0.5 * (bias_resolution_2['reco_energy_low'] + bias_resolution_2['reco_energy_high']).to_value(u.TeV),
        bias_resolution_2['bias']+1,  # not sure ...
        xerr=0.5 * (bias_resolution_2['reco_energy_high'] - bias_resolution_2['reco_energy_low']).to_value(u.TeV),
        ls='',
        label=label_protopipe_previous,
        color=color_protopipe_previous
    )

# ED
if load_EventDisplay:
    y, edges = ED_performance["Ebias"].to_numpy()
    yerr = ED_performance["Ebias"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=ED_label,
                 color=color_EventDisplay
                )

# Style settings
plt.xscale('log')
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Energy bias")
plt.xlim(0.009, 500)
plt.grid(which="both", axis="both", visible=True)
plt.legend(loc="best")
plt.title(production)

None # to remove clutter by mpl objects

### Background rate
[back to top](#Table-of-contents)

In [2]:
def plot_background_rate(file, ax=None, label=None, color=None, ls=''):
    
    from pyirf.utils import cone_solid_angle
    
    ax = plt.gca() if ax is None else ax
    
    bg_rate = QTable.read(file, hdu='BACKGROUND')[0]

    reco_bins = np.append(bg_rate['ENERG_LO'], bg_rate['ENERG_HI'][-1])

    # first fov bin, [0, 1] deg
    fov_bin = 0
    rate_bin = bg_rate['BKG'].T[:, fov_bin]

    # interpolate theta cut for given e reco bin
    e_center_bg = 0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI'])
    e_center_theta = 0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI'])
    theta_cut = np.interp(e_center_bg, e_center_theta, rad_max['RAD_MAX'].T[:, 0])

    # undo normalization
    rate_bin *= cone_solid_angle(theta_cut)
    rate_bin *= np.diff(reco_bins)
    plt.errorbar(
        0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI']).to_value(u.TeV)[1:-1],
        rate_bin.to_value(1 / u.s)[1:-1],
        xerr=np.diff(reco_bins).to_value(u.TeV)[1:-1] / 2,
        ls=ls,
        label=label,
        color=color
    )
    
    return ax

In [ ]:
fig = plt.figure(figsize=get_fig_size(ratio=4/3., scale=scale))

# protopipe
plot_background_rate(protopipe_file,
                     ax=plt.gca(),
                     label=label_protopipe_current,
                     color=color_protopipe_current)

if load_protopipe_previous:
    plot_background_rate(protopipe_file_2,
                         ax=plt.gca(),
                         label=label_protopipe_previous,
                         color=color_protopipe_previous)

# ED
if load_EventDisplay:
    y, edges = ED_performance["BGRate"].to_numpy()
    yerr = ED_performance["BGRate"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=ED_label,
                 color=color_EventDisplay)


# MARS
if load_CTAMARS:
    y, edges = MARS_performance["BGRate"].to_numpy()
    yerr = MARS_performance["BGRate"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    plt.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=MARS_label,
                 color=color_CTAMARS)


# Style settings
plt.xscale("log")
plt.xlim(1.e-2, 3.e2)
plt.ylim(1.e-11, 1.e0)
plt.xlabel("Reconstructed energy [TeV]")
unit = u.Unit('s-1 TeV-1')
plt.ylabel(f"Background rate / ({unit.to_string('latex')}) ")
plt.grid(which="both", axis="both", visible=True)
plt.legend(loc="lower left")
plt.title(production)
plt.yscale('log')

None # to remove clutter by mpl objects